# Matrix Analysis 2023 - EE312

## Week 5 - QR DECOMPOSITION
[LTS2](https://lts2.epfl.ch)

### Objectives
The goal of this week's exercise session is to study some aspects of the QR decomposition and its connections to projections and inverse.

Let us consider a square $n\times n$ real matrix $A$ having linearly independent columns (NB: QR factorization is applicable to rectangular matrices, we consider square matrices for simplification). The QR decomposition of $A$ is $A=QR$ where $Q$ is an orthogonal matrix and $R$ and upper-triangular matrix.

We will study two methods that can be used to compute the QR decomposition of a matrix and an application.

# 1. Gram-Schmidt orthogonalization

Reminder:
- The projection operator of a vector $v$ on another $u$ is given by $P_u v = \frac{\langle u,v \rangle}{\langle u,u \rangle}u$.
- Gram-Schmidt orthogonalization process of a basis $(v_1, v_2, ..., v_{n})$ produces an orthormal basis $(e_1, e_2, ..., e_{n})$ as follows:

$u_1 = v_1$, $e_1 = \frac{u_1}{\|u_1\|}$

$u_2 = v_2 - P_{u_1}v_2$, $e_2 = \frac{u_2}{\|u_2\|}$

$u_3 = v_3 - P_{u_2}v_3 - P_{u_1}v_3$, $e_3 = \frac{u_3}{\|u_3\|}$

...

$u_k = v_k - \sum_{j=1}^{k-1} P_{u_j}v_{k}$, $e_k = \frac{u_k}{\|u_k\|}$



## 1.1 
We perform the QR decomposition of $A$ by applying Gram-Schmidt to its column vectors of $A$ (denoted by $a_k, k=1,...n)$. $Q$ is made of the column vectors $e_k$, i.e. $Q=(e_1|e_2|...|e_{n})$. Let us denote by $r_{jk}$ the coefficients of $R$.

Prove that $r_{kk} = \|u_k\|$, $r_{jk} = <e_j, a_k>$ for $j<k$ and $r_{jk} = 0$ for $j>k$.


*Answer*: Using the Gram-Schmidt formula, we can write:
$a_k = u_k + \sum_{j=1}^{k-1} P_{u_j}a_{k} = u_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|^2}u_j = \|u_k\|e_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|^2}\|u_j\|e_j = \|u_k\|e_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|}e_j = \|u_k\|e_k + \sum_{j=1}^{k-1}<e_j, a_k>e_j $

Using the definition of $Q$ and $R$ we also have $a_k = \sum_{j=1}^n r_{jk} e_j$. Using the equality above, this proves that $r_{kk} = \|u_k\|$, $r_{jk} = <e_j, a_k>$ for $j<k$ and $r_{jk} = 0$ for $j>k$.




## 1.2
Implement a function that performs the QR decomposition of a square $n\times n$ matrix using the Gram-Schmidt orthogonalization process

In [1]:
import numpy as np
def qr_decomp_gs(A):
    n = A.shape[0]
    Q = np.zeros((n,n))
    R = np.zeros((n,n))
    U = np.zeros((n,n))
    U[:, 0] = A[:, 0]
    Q[:, 0] = A[:, 0]/np.linalg.norm(A[:, 0])
    R[0, 0] = np.linalg.norm(A[:, 0])
    for k in np.arange(1, n):
        acc = np.zeros((n))
        for j in np.arange(0, k):
            R[j, k] = np.dot(A[:, k], Q[:, j])
            acc = acc + (np.dot(A[:, k], U[:, j])/np.dot(U[:, j], U[:, j]))*U[:, j]

        U[:, k] = A[:, k] - acc
        Q[:, k] = U[:, k]/np.linalg.norm(U[:, k])
        R[k, k] = np.linalg.norm(U[:, k])
            
    return Q,R

## 1.3
Using the properties of Q, write a routine that can solve a linear system $Ax=b$ using the QR factorization of $A$ ($A$ being a $n\times n$ real matrix).
What property do the coefficients of $R$ need to satisfy for the solution to exist ?

*Answer:* We have $Ax = b$ and $A=QR$ with $Q$ orthogonal hence $Q^TQRx = Q^Tb$ i.e. $Rx=Q^Tb$. 

Given $R$ is upper triangular, we do not need to perform compute $R^{-1}$ explicitely and the system can be solved using backsubstitution. 
Since we divide by $r_{kk}$ to get the $x_k$ values, those diagonal coefficients have to be non-zero.

In [2]:
def solve(A, b):
    n = A.shape[0]
    Q, R = qr_decomp_gs(A)
    c = Q.T@b
    x = np.zeros(n)
    x[n-1] = c[n-1]/R[n-1, n-1]
    for k in np.arange(n-2, -1, -1):
        s = np.dot(R[k, k+1:], x[k+1:])
        x[k] = (c[k] - s)/R[k,k]
    return x

In [7]:
# Small numerical example
A=np.array([[12,-51,4],[6,167,-68],[-4,24,-41]])
b=np.array([1, -1, 1])

In [4]:
x = solve(A, b)

In [5]:
A@x

array([ 1., -1.,  1.])

## 1.4
Consider the matrix 
$B=\begin{pmatrix}1 & 1 & 1 \\ \varepsilon & 0 & 0\\ 0 & \varepsilon & 0\end{pmatrix}$, with $\varepsilon$ being small enough (typically $10^{-8}$ or smaller). In that case, is the method used to compute Q and R still reliable (and why) ?

In [2]:
eps=1e-8
B=np.array([[1,1,1],[eps,0,0],[0,eps,0]])

In [3]:
B

array([[1.e+00, 1.e+00, 1.e+00],
       [1.e-08, 0.e+00, 0.e+00],
       [0.e+00, 1.e-08, 0.e+00]])

In [43]:
QB,RB = qr_decomp_gs(B)

In [44]:
QB@QB.T

array([[ 1.0e+00,  1.0e-08,  0.0e+00],
       [ 1.0e-08,  1.5e+00, -5.0e-01],
       [ 0.0e+00, -5.0e-01,  5.0e-01]])

In [45]:
QB@RB-B

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

We still have $QR=B$, but $Q$ is no longer orthogonal ! This is due to the numerical precision of the computer, that rounds off the norm of the first column of $B$ to 1, when it should be $\sqrt{1+\varepsilon^2}$.

In [11]:
np.linalg.norm(B[:,0])

1.0

# 2. Householder reflections

We will now study and implement another method to perform the QR decomposition that is more resistant to the numerical issues mentioned above.

## 2.1 
Let $v$ be a unit vector and the associated transform $H_v=I-2vv^T$. Prove that $H_v$ is orthogonal and that it preserves the norm. What is the effect of $H_v$ on a vector $u$ orthogonal to $v$ ?

*Answer:* $H_vH_v^T = (I - 2vv^T)(I - 2vv^T)^T = (I - 2vv^T)(I - 2vv^T) = I -4vv^T + 4v(v^Tv)v^T$

Since $v$ is a unit vector, $v^Tv=1$ i.e. $H_vH_v^T = I - 4vv^T + 4vv^T = I$.

$\|H_vu\|^2 = (H_vu)^TH_vu = u^TH_v^TH_vu =  u^Tu = \|u\|^2$, $H_v$, as any orthogonal transform preserves norm. 

If $u$ is orthogonal to $v$, we have $v^Tu=0$.
$H_vu = u - 2 vv^Tu = u$. $H_v$ does not modify vectors orthogonal to $v$.

## 2.2 
What is the effect of $H_v$ on any vector $u$

*Answer:* $H_vu = u - 2vv^Tu = u - 2<v,u>v$

The term $<v,u>v$ is in fact the orthogonal projection of $u$ on $v$ (remember $<v,v>=\|v\|=1)$. If we draw the effect of $H_v$ on $u$, it yields the reflection of $u$ around $v^\perp$.

![householder](images/householder.png)

## 2.3
The goal is now to design $n$ reflection operators $H_{v_1}, H_{v_2}, ..., H_{v_n}$ s.t. we have
$$
H_{v_n}...H_{v_2}H_{v_1}A = R,
$$
where $R$ is an upper triangular matrix. 

The reflection operations are meant to be "progressive", i.e. $H_{v_k}...H_{v_1}A$ has its $k$ first columns upper triangular.

- Find $v_1$ and $\alpha$ s.t. $H_{v_1}a_1 = \alpha e_1$, where $a_1$ is the first column of $A$ and $e_1 = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix}$.

*Answer:* $H_{v_1}a_1 = (I -2v_1v_1^T)a_1 = \alpha e_1$, i.e. $a_1 - 2v_1v_1^Ta_1 = \alpha e_1$, 
and finally $a_1 - \alpha e_1 = 2 <v_1, a_1> v_1$, which implies that $v_1$ is colinear to $a_1 - \alpha e_1$.
Remembering that $H_{v_1}$ preserves norm, we have $\|H_{v_1}a_1\| = \|a_1\| = |\alpha|$.
Therefore $v_1 = \frac{1}{\|a_1 \pm \|a_1\|e_1\|}(a_1 \pm \|a_1\|e_1)$ 

In [12]:
u=A[:,0] - np.linalg.norm(A[:,0])*np.array([1, 0, 0])
v = u/np.linalg.norm(u)

In [13]:
np.linalg.norm(A[:,0])

14.0

In [14]:
H=np.eye(3)- 2*np.kron(v, v).reshape((3,3))

In [15]:
H@A[:, 0]

array([ 1.4000000e+01, -4.4408921e-16,  8.8817842e-16])

- Let us now generalize the previous step to find $H_{v_k}$. Given a vector $x = \begin{pmatrix}x^U \\ x^L\end{pmatrix}$ with $x^U \in\mathbb{R}^k$
and $x^L\in\mathbb{R}^{n-k}$. 

Find a vector $v_k$ s.t. $H_{v_k}x = \begin{pmatrix}x^U \\ \alpha e_1^L \end{pmatrix}$, where $\alpha\in\mathbb{R}$ and 
$e_1^L = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix} \in \mathbb{R}^{n-k}$. 

It might be of help to write $v_k=\begin{pmatrix}v_k^U\\v_k^L\end{pmatrix}$, with $v_k^U \in\mathbb{R}^k$
and $v_k^L\in\mathbb{R}^{n-k}$. 

*Answer:*  Since we have $H_{v_k} = I - 2v_kv_k^T = I - 2\begin{pmatrix}v_k^U\\v_k^L\end{pmatrix}\begin{pmatrix}(v_k^U)^T & (v_k^L)^T\end{pmatrix}$, which leads to $H_{v_k} = I - 2\begin{pmatrix}v_k^U(v_k^U)^T & v_k^U(v_k^L)^T \\ (v_k^U)^Tv_k^L & v_k^L(v_k^L)^T\end{pmatrix}$

$H_{v_k}x = \begin{pmatrix}x^U \\ x^L\end{pmatrix} - 2\begin{pmatrix}v_k^U(v_k^U)^T & v_k^U(v_k^L)^T \\ (v_k^U)^Tv_k^L & v_k^L(v_k^L)^T\end{pmatrix}\begin{pmatrix}x^U \\ x^L\end{pmatrix}$

If we want the upper part of $x$ to be preserved, it is relatively obvious that $v_k^U = 0$ will be needed, which leads to
$H_{v_k}x = \begin{pmatrix}x^U \\ x^L\end{pmatrix} - 2\begin{pmatrix}0 & 0 \\ 0 & v_k^L(v_k^L)^T\end{pmatrix}\begin{pmatrix}x^U \\ x^L\end{pmatrix}$.

The lower part yields: $x^L - 2 v_k^L(v_k^L)^Tx^L = x^L - 2<v_k^L, x^L>v_k^L  = \alpha e_1^L$. As for $v_1$, we see that $v_k$ is colinear to $x^L - \alpha e_1^L$, and the norm preservation property yields $\alpha = \pm\|x^L\|$.

Finally, $v_k^L = \frac{1}{\|x^L \pm \|x^L\|e_1^L \|}(x^L \pm \|x^L\|e_1^L)$

## 2.4
We now have all we need to compute the $H_{v_k}$ matrices. Implement the QR decomposition using the above results. 
You should have found that the sign of $\alpha$ can be either positive or negative. Choose $\alpha$ to have the sign opposite to the one of $x^L_k$. (Use the `numpy.sign`function).

Try your implementation on the $B$ matrix seen in ex. 1. What is the advantage of this implementation ?

In [4]:
def qr_decomp_hh(A):
    n = A.shape[0]
    E = np.eye(n)
    Q = np.eye(n)
    R = A.copy()
    for k in np.arange(0, n):
        u = np.zeros(n)
        s = np.sign(R[k,k])
        u[k:] = R[k:,k] + s*np.linalg.norm(R[k:,k])*E[k:, k]
        v = u/np.linalg.norm(u)
        H = E - 2*np.outer(v, v)
        R = H@R
        Q = H@Q
    return Q.T,R # in this case we need the 'inverse' of Q

In [8]:
qh, rh = qr_decomp_hh(A)

In [9]:
qh@rh -A

array([[-3.55271368e-15,  4.26325641e-14, -1.59872116e-14],
       [ 0.00000000e+00, -8.52651283e-14,  4.26325641e-14],
       [ 4.44089210e-16, -3.55271368e-15,  0.00000000e+00]])

In [10]:
qh@qh.T

array([[ 1.00000000e+00,  1.26634814e-16,  1.11022302e-16],
       [ 1.26634814e-16,  1.00000000e+00, -4.85722573e-17],
       [ 1.11022302e-16, -4.85722573e-17,  1.00000000e+00]])

In [11]:
qh, rh = qr_decomp_hh(B)

In [12]:
qh@qh.T

array([[ 1.00000000e+00,  1.15805286e-23, -4.96308368e-24],
       [ 1.15805286e-23,  1.00000000e+00,  4.44089210e-16],
       [-4.96308368e-24,  4.44089210e-16,  1.00000000e+00]])

In [13]:
qh@rh-B

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.45050537e-23,  1.15805286e-23],
       [ 0.00000000e+00, -4.96308368e-24, -4.96308368e-24]])

The QR factorization using the Householder reflections is now giving correct results on $B$. This seems a better choice in terms of numerical stability. 